In [4]:
import pandas as pd
import random
import time
import json
import requests
import os


You can find API from [7-11website](https://www.7eleven.co.th/find-store) by use inspector

In [33]:
def get_seven_store(lat, long):
    url = "https://7-11API"
    payload = {"latitude": lat, "longitude": long}
    headers = {'Accept': 'application/json, text/plain, */*'}
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()
    else: 
        print(f"Error: Received status code {response.status_code}")
        return None

Read tambon df and use TA_ID to scrape location based on Thai Tambon

In [ ]:
selected_province_list = ["กรุงเทพมหานคร"]
tambon_lat_long_df = pd.read_excel('tambon.xlsx')
tambon_lat_long_df = tambon_lat_long_df[tambon_lat_long_df.CHANGWAT_T.isin(selected_province_list)]
tambon_lat_long_df = tambon_lat_long_df[["TA_ID", "TAMBON_T", "LAT", "LONG"]].drop_duplicates(subset='TA_ID', keep="first")
tambon_lat_long_df = tambon_lat_long_df.sort_values(by=['TA_ID']).reset_index()
print(tambon_lat_long_df.head())

Get Data

In [ ]:
RESULT_DIRECTORY = "7-11data"
print("Total api calls:", len(tambon_lat_long_df))
for index, row in tambon_lat_long_df.iterrows():
    print(index, row['TA_ID'], row['TAMBON_T'], row['LAT'], row['LONG'])
    json_data = get_seven_store(row['LAT'], row['LONG'])
    filename = f"{RESULT_DIRECTORY}/7-11raw/{index}_{int(row['TA_ID'])}.json"
    with open(filename, 'w', encoding='utf-8') as outfile: json.dump(json_data, outfile, ensure_ascii=False)
    time.sleep(random.randint(30, 90)) #random sleep time !!! do not flood the API

Aggregate json files to Dataframe

In [77]:
input_folder = '7-11data/7-11raw'
output_file = '7-11data/7-11Bangkok.csv'

all_data = []

for filename in os.listdir(input_folder):
    if filename.endswith('.json'):   # Process only JSON files
        input_path = os.path.join(input_folder, filename)

        with open(input_path, 'r', encoding='utf-8') as infile:
            original_dat = json.load(infile)
            data_array = original_data.get("data", [])   # Extract the 'data' key
            df = pd.json_normalize(data_array)
            all_data.append(df)
            
aggregated_df = pd.concat(all_data, ignore_index=True)
aggregated_df.to_csv(output_file, index=False)

Example Data

In [37]:
data = pd.read_csv("7-11data/7-11Bangkok.csv")
data.head()

,id,lat,lng,code,name,address,subdistrict,district,province,postcode,products,lineOA,tel,distance,renovate,direction
0,1461,13.752521,100.490451,1461,หน้าพระลาน,ถ.หน้าพระลาน แขวงพระบรมมหาราชวัง เขตพระนคร กรุ...,พระบรมมหาราชวัง,พระนคร,กรุงเทพมหานคร,10200,"[{'id': 6, 'code': 'AC', 'name': 'ออลล์คาเฟ่',...",NaN,"02-225-4763-4,091-002-1461",238.161324,False,https://www.google.com/maps/search/?api=1&quer...
1,3046,13.752902,100.496298,3046,คลองหลอด,ถ.อัษฎางค์ แขวงศาลเจ้าพ่อเสือ เขตพระนคร กรุงเท...,ศาลเจ้าพ่อเสือ,พระนคร,กรุงเทพมหานคร,10200,"[{'id': 13, 'code': 'VF', 'name': 'ผักและผลไม้...",@252gvrwu,"02-622-3212-3,091-002-3046",510.691555,False,https://www.google.com/maps/search/?api=1&quer...
2,96,13.756120,100.489528,96,ท่าพระจันทร์,ถ.มหาราช แขวงพระบรมมหาราชวัง เขตพระนคร กรุงเทพ...,พระบรมมหาราชวัง,พระนคร,กรุงเทพมหานคร,10200,"[{'id': 13, 'code': 'VF', 'name': 'ผักและผลไม้...",@531fhegh,"02-221-0261,091-002-0096",629.553723,False,https://www.google.com/maps/search/?api=1&quer...
3,1559,13.750043,100.498147,1559,กระทรวงมหาดไทย,ถ.เฟื่องนคร แขวงวัดราชบพิธ เขตพระนคร กรุงเทพมห...,วัดราชบพิธ,พระนคร,กรุงเทพมหานคร,10200,"[{'id': 13, 'code': 'VF', 'name': 'ผักและผลไม้...",@304gpgii,"02-622-3386-7,091-002-1559",673.148811,False,https://www.google.com/maps/search/?api=1&quer...
4,3801,13.754875,100.497318,3801,บุญศิริ,แขวงศาลเจ้าพ่อเสือ เขตพระนคร กรุงเทพมหานคร,ศาลเจ้าพ่อเสือ,พระนคร,กรุงเทพมหานคร,10200,"[{'id': 13, 'code': 'VF', 'name': 'ผักและผลไม้...",@868vfdpo,"02-622-0027-8,091-002-3801",718.833192,False,https://www.google.com/maps/search/?api=1&quer...
